Testing document for pyGSLModel

In [ ]:
# Downloading pruned model
from pyGSLModel.src.pyGSLModel import download_GSL_model

model = download_GSL_model()

In [ ]:
import pandas as pd
# Making a dummy dataframe
d = {
    "Gene" : ["B4GALNT1", "ST3GAL5", "ST8SIA1"],
    "Sample_1" : [10,5,1],
    "Sample_2" : [1,5,10]
}

test_df = pd.DataFrame(d)

test_df = test_df.set_index("Gene")

test_df

In [ ]:
from pyGSLModel.src.pyGSLModel import iMAT_integrate

imat_test = iMAT_integrate(model, test_df)